In [1]:
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

In [2]:
load_dotenv()

True

In [3]:
# Get the key from env variable
groq_key = os.getenv("GROQ_API_KEY")

In [4]:
model = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key
)

In [5]:
# define the state of the blog
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [14]:
# define function to generate a blog outline 

def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state["title"]

    # form a prompt 
    prompt = f"Create a blog outline for the following title: {title}"

    # ask that question to the LLM using the OpenAI-compatible SDK
    response = model.chat.completions.create(
        model="llama3-70b-8192",  # or any other supported Groq model
        messages=[{"role": "user", "content": prompt}]
    )

    # extract the outline
    outline = response.choices[0].message.content

    # update the state with the outline 
    state["outline"] = outline

    return state

In [18]:
# define function to generate a blog 

def create_blog(state: BlogState) -> BlogState:

    title = state["title"]
    outline = state["outline"]

    prompt = f"Create a blog for the following title: {title} and outline: {outline}"

    # ask that question to the LLM using the OpenAI-compatible SDK
    response = model.chat.completions.create(
        model="llama3-70b-8192",  # or any other supported Groq model
        messages=[{"role": "user", "content": prompt}]
    )

    # extract the outline
    content = response.choices[0].message.content

    state["content"] = content

    return state 

In [19]:
# create our graph 
graph = StateGraph(BlogState)

# create nodes 
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)

# create edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)

# compile the graph
workflow = graph.compile()


In [23]:
intial_state = {'title': 'World in 2030'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'World in 2030', 'outline': 'Here is a suggested outline for a blog post on the title "World in 2030":\n\n**I. Introduction**\n\n* Brief overview of the rapid pace of technological advancements and global changes\n* Thesis statement: By 2030, the world will have undergone significant transformations, driven by technological innovations, shifting global power dynamics, and environmental concerns.\n\n**II. Technological Advancements**\n\n* Subheading: The Rise of AI, Robotics, and Quantum Computing\n\t+ Discussion of the expected advancements in artificial intelligence, robotics, and quantum computing\n\t+ Examples of potential applications in industries such as healthcare, finance, and education\n* Subheading: The Internet of Things (IoT) and 5G Networks\n\t+ Explanation of how IoT and 5G networks will enable greater connectivity and data exchange\n\t+ Potential implications for smart cities, transportation, and energy management\n\n**III. Global Economy and Power Dynamics**\n

In [24]:
print(final_state['outline'])

Here is a suggested outline for a blog post on the title "World in 2030":

**I. Introduction**

* Brief overview of the rapid pace of technological advancements and global changes
* Thesis statement: By 2030, the world will have undergone significant transformations, driven by technological innovations, shifting global power dynamics, and environmental concerns.

**II. Technological Advancements**

* Subheading: The Rise of AI, Robotics, and Quantum Computing
	+ Discussion of the expected advancements in artificial intelligence, robotics, and quantum computing
	+ Examples of potential applications in industries such as healthcare, finance, and education
* Subheading: The Internet of Things (IoT) and 5G Networks
	+ Explanation of how IoT and 5G networks will enable greater connectivity and data exchange
	+ Potential implications for smart cities, transportation, and energy management

**III. Global Economy and Power Dynamics**

* Subheading: The Rise of Emerging Markets and Shift in Glo

In [25]:
print(final_state['content'])

**The World in 2030: A Future of Transformation and Innovation**

**I. Introduction**

In the past few decades, the world has witnessed unprecedented technological advancements, global shifts, and environmental concerns. As we look towards the future, it's clear that the next decade will bring even more rapid and profound changes. By 2030, the world will have undergone significant transformations, driven by technological innovations, shifting global power dynamics, and environmental concerns. In this blog post, we'll explore the potential future of our world and the implications for humanity.

**II. Technological Advancements**

### The Rise of AI, Robotics, and Quantum Computing

Artificial intelligence, robotics, and quantum computing are expected to make tremendous progress in the next decade. AI will become even more integrated into our daily lives, from healthcare to finance to education. For instance, AI-powered diagnosis will revolutionize healthcare, enabling quicker and more a